In [68]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import snowflake.connector
from sklearn.impute import SimpleImputer
import numpy as np
import os
from dotenv import load_dotenv




In [69]:
load_dotenv()
# print()
# Database connection parameters
hostname = os.getenv("HOSTNAME")
database = os.getenv("DATABASE")
username = os.getenv("USER_NAME")
pwd = os.getenv("PGPASS")
port_id = os.getenv("PORTID")


print(hostname)
print(database)
print(username)
print(pwd)
print(port_id)



# Creating SQLAlchemy engine
engine = create_engine(f'postgresql://{username}:{pwd}@{hostname}/{database}')


localhost
ecommerce
postgres
33386
5432


In [70]:
# Fetch the environment variables
snow_user = os.getenv("SNOW_USER")
snow_password = os.getenv("SNOW_PASSWORD")
account = os.getenv("ACCOUNT")
warehouse = os.getenv("WAREHOUSE")
snow_database = os.getenv("SNOW_DATABASE")
schema = os.getenv("SCHEMA")

# You can now use these variables in your application
print("Username:", snow_user)
print("Password:", snow_password)
print("Account:", account)
print("Warehouse:", warehouse)
print("Database:", snow_database)
print("Schema:", schema)

# Establish connection using the environment variables
snowflake_connection = snowflake.connector.connect(
    user=snow_user,
    password=snow_password,
    account=account,
    warehouse=warehouse,
    database=snow_database,
    schema=schema
)

print(snowflake_connection)


# snowflake_connection.cursor().execute("DROP SCHEMA  testschema_mg")

Username: kamanlimbu
Password: Kaman@33386
Account: yh31700.ap-south-1
Warehouse: ECOMMERCE_WH
Database: ECOMMERCE
Schema: ECOMMERSE_SCHEMA


In [44]:


def extract():
    try:
        # Connecting to PostgreSQL database
        connection = psycopg2.connect(
            host=hostname, 
            dbname=database, 
            user=username, 
            password=pwd, 
            port=port_id
        )
        curr = connection.cursor()
        connection.set_session(autocommit=True)

        query = """
            SELECT DISTINCT(table_name) as tableNAME
            FROM
                information_schema.columns
            WHERE
                table_schema = 'public'
            ORDER BY
                table_name
        """
        curr.execute(query)
        result = curr.fetchall()

        for tab in result:
            # print(tab[0])
            df = pd.read_sql_query(f'SELECT * FROM {tab[0]}', engine)
            transform(df, tab[0])

    except psycopg2.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Other error: {e}")
    finally:
        if curr:
            curr.close()
        if connection:
            connection.close()


def transform(df, tab):
    # Implement the actual load functionality here
    print("Starting transformation..1..2..3!!")
    print("Table Name:", tab)
    print("Before rows:", df.shape)

    numerical_columns = df.select_dtypes(include=[np.number]).columns
    text_columns = df.select_dtypes(include=[object]).columns
    date_columns = df.select_dtypes(include=[np.datetime64]).columns

    if not df.empty and len(numerical_columns) > 0:
    # Impute missing values in numerical columns
        numerical_imputer = SimpleImputer(strategy='mean')
        df[numerical_columns] = numerical_imputer.fit_transform(df[numerical_columns])
    else:
        print("No numerical columns to impute.")

    if not df.empty and len(text_columns) > 0:
    # Impute missing values in text columns
        df[text_columns] = df[text_columns].fillna('Not Available')
        # text_imputer = SimpleImputer(strategy='missing_values', fill_value='Not Available')
        # df[text_columns] = text_imputer.fit_transform(df[text_columns])
    else:
        print("No text columns to impute.")

    if not df.empty and len(date_columns) > 0:
    # Impute missing values in date columns using forward fill
        df[date_columns] = df[date_columns].fillna(method='ffill')
        # If any NaT values remain (e.g., if the first value was NaT), use backward fill
        df[date_columns] = df[date_columns].fillna(method='bfill')
    else:
        print("No date columns to impute.")

    display(df[df.isna().any(axis=1)])
    display(df.columns.dtype)

    updated_dataframe = pd.DataFrame(df)
    updated_dataframe.drop_duplicates()
    print("Table Name:", tab)
    display(updated_dataframe.head(10))
    load(updated_dataframe, tab)


def load(dataframe, tab):
    print("Loading Data in Snowflake data warehouse...")
    print("1....2.....3.....")
    print(f"Create {tab} table in SnowFlake")

    dtype_mapping = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'STRING'
    }

    create_table_sql = f"""
    CREATE OR REPLACE TABLE {tab} (
    {', '.join([f'{col} {dtype_mapping[str(dtype)]}' for col, dtype in dataframe.dtypes.items()])}
    );
    """
    snowflake_connection.cursor().execute(create_table_sql)

    # Batch size
    batch_size = 200000 // len(dataframe.columns)  # Adjust batch size based on the number of columns

    # Insert data into Snowflake table in batches
    insert_sql = f"INSERT INTO {tab} ({', '.join(dataframe.columns)}) VALUES ({', '.join(['%s'] * len(dataframe.columns))})"
    for start in range(0, len(dataframe), batch_size):
        batch_data = [tuple(row) for row in dataframe.iloc[start:start + batch_size].itertuples(index=False, name=None)]
        with snowflake_connection.cursor() as cursor:
            cursor.executemany(insert_sql, batch_data)
    
    print(f"Data loaded into {tab} successfully!")

extract()




Starting transformation..1..2..3!!
Table Name: customer
Before rows: (99441, 5)
No date columns to impute.


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


dtype('O')

Table Name: customer


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409.0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790.0,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151.0,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775.0,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056.0,campinas,SP
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254.0,jaragua do sul,SC
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534.0,sao paulo,SP
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182.0,timoteo,MG
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560.0,curitiba,PR
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575.0,belo horizonte,MG


...

,seller_id,seller_zip_code_prefix,seller_city,seller_state


dtype('O')

Table Name: sellers


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023.0,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844.0,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031.0,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195.0,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914.0,braganca paulista,SP
5,c240c4061717ac1806ae6ee72be3533b,20920.0,rio de janeiro,RJ
6,e49c26c3edfa46d227d5121a6b6e4d37,55325.0,brejao,PE
7,1b938a7ec6ac5061a66a3766e0e75f90,16304.0,penapolis,SP
8,768a86e36ad6aae3d03ee3c6433d61df,1529.0,sao paulo,SP
9,ccc4bbb5f32a6ab2b7066a4130f114e3,80310.0,curitiba,PR


Loading Data in Snowflake data warehouse...
1....2.....3.....
Create sellers table in SnowFlake
Data loaded into sellers successfully!


dtype('float64')